In [1]:
import pandas as pd
import re

In [2]:
users = pd.read_csv('./users/19.02.19-12:11.csv', index_col=0)
sub = pd.read_csv('./sub/19.02.19-12:11.csv', index_col=0)

/home/azat/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/azat/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/azat/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def age_filter(user, birth_year_list, is_include_without_bdate=False):
    if user['bdate']:
        year = user['bdate'].split('.')[-1]
        if len(year) == 4:
            return int(year) in birth_year_list
        else:
            return is_include_without_bdate
    else:
        return is_include_without_bdate
    
def sex_filter(user, sex, is_include_without_sex=False):
    if user['sex']:
        return int(user['sex']) == sex
    else:
        return is_include_without_sex

def occupation_filter(user, occupation_list=['work', 'university', 'school'], is_include_without_occupation=False):
    if user['type_occupation']:
        return user['type_occupation'] in occupation_list
    else:
        return is_include_without_occupation

def smoking_filter(user, smoking_barrier, is_include_without_smoking=True):
    """smoking barrier:    
        1 — резко негативное;
        2 — негативное;
        3 — компромиссное;
        4 — нейтральное;
        5 — положительное.
    """
    if user['smoking']:
        return user['smoking'] <= smoking_barrier
    else:
        is_include_without_smoking
        
def gimnazia_filter(user, is_include_without_school=False):
    if user['school_name']:
        school_name = user['school_name']
        s = re.match(r'(.*гимназия.*|.*лицей.*)', school_name.lower())
        return bool(s)
    else:
        return is_include_without_school

def relation_filter(user, list_posible_relations, is_include_without_relation=False):
    """
    1 — не женат/не замужем;
    2 — есть друг/есть подруга;
    3 — помолвлен/помолвлена;
    4 — женат/замужем;
    5 — всё сложно;
    6 — в активном поиске;
    7 — влюблён/влюблена;
    8 — в гражданском браке;
    0 — не указано. 
    """
    if user['relation']:
        rel = int(user['relation'])
        if rel == 0:
            return is_include_without_relation
        return rel in list_posible_relations
    else:
        return is_include_without_relation
    
def political_filter(user, list_posible_political, is_include_without_political=False):
    """
    1 — коммунистические;
    2 — социалистические;
    3 — умеренные;
    4 — либеральные;
    5 — консервативные;
    6 — монархические;
    7 — ультраконсервативные;
    8 — индифферентные;
    9 — либертарианские.
    """
    if user['political']:
        pol = int(user['political'])
        return pol in list_posible_political
    else:
        return is_include_without_political

def people_main_filter(user, list_posible_main, is_include_without=False):
    """
    1 — ум и креативность;
    2 — доброта и честность;
    3 — красота и здоровье;
    4 — власть и богатство;
    5 — смелость и упорство;
    6 — юмор и жизнелюбие.
    """
    if user['people_main']:
        pm = int(user['people_main'])
        return pm in list_posible_main
    else:
        is_include_without

def military_filter(user, usersub, need_value=True, is_include_without=True):
    if user['military']:
        m = bool(user['military'])
        return m == need_value
    else:
        is_include_without

def life_main_filter(user, list_posible_life, is_include_without=False):
    """
    1 — семья и дети;
    2 — карьера и деньги;
    3 — развлечения и отдых;
    4 — наука и исследования;
    5 — совершенствование мира;
    6 — саморазвитие;
    7 — красота и искусство;
    8 — слава и влияние;
    """
    if user['life_main']:
        pm = int(user['life_main'])
        return pm in list_posible_main
    else:
        is_include_without

def graduation_filter(user, already_grad=True, is_include_without=False):
    if user['graduation']:
        year = int(user['graduation'])
        if already_grad:
            return 2019 >= year  
        else:
            return 2019 < year
    else:
        return is_include_without

In [4]:
def nauchpop_filter(user, user_sub, threshold=6):
    s = """.*научн.*|.*наука.*|.*исследован.*|Киберленинка|.*ANTROPOGENEZ.*|SciOne|.*открытия.*|.*TEDX.*|.*биоло.*|.*постнаука.*|.*n+1.*|.*метеорология.*|VSauce|GEO|Prolegarium|Skinner Show|Utopia Show|Чуть-Чуть о Науке|Постоянная Планка|TechKnowledge|ZHITKOV|Valeriy Sobolev|LOONY Science|SciTeam|Михаил Лидин|TrashSmash|Smash|SciOne|SciOne LIVE|Артур Шарифов|Научпок|Alexander Obraz|Джо Чиз|kvashenov|Премия имени Гарри Гудини|ТОПЛЕС|QWERTY|360.Наука|Sinus|Чердак: наука, технологии, будущее|Alice Lafox|Научно-популярный хаб Lafox.Net|CHILDRENscience|xRealMansChannelx|Молния Наука|mythologyChanel|Здравамир|Высшая школа методологии|Игры Разума|Андрей Курпатов|99thoughtz|Iskusstvo Znat|SciTopus|Это Работает|AlexTranslations|Алан Грант — Наука с Публицистикой|InsaneSpaceMuffin|In Cor Cadit|Джонник|DeeAFilm Studio|ПЛЮШКИН ДОМ|Ученая Обезьяна|Невидимый Розовый Единорог|SpaceWhale|Vert Dider|Vert Dider|VoicePower|Darvin Studio|mobius band|Фактор понимания|Ламповая Наука|BuzzFeed Russia|Научкэп|Veritasium Russia|Gloomy Voice|Mike Piligrim|ТО Скарф|Frontal Переводы|Журнал СРЕЗ|Двое в лодке|FRED DUB|sci-scope|Потребление Идей|Смысловые пятиминутки|Мелтин|Dagon|Мудреныч|Чорна Птаха|Мозговик|TED|rutedtalks|FEDORISTED|Егор Речи|НаучПоп|Общество Скептиков|Центр Архэ|Курилка Гутенберга|15x4 Talks|НаукаPRO|Познаем Общаясь|Прогресс-Школа|Зануда|Лаборатория Научных Видео|АРХЭология знаний|КритМышь|Фонд Эволюция|Фонд Траектория|XX2 ВЕК|Лекториум|Set Up|ПостНаука|KL10CH|Фенист|Антикафе Кочерга|Noosphere Studio|Группа "Думай"|Geek Picnic|Учёба.ру|Парк науки|Alex Gapak|Политехнического музея Лекторий|Российская государственная библиотека для молодежи (РГБМ)|BrainUp|FutureBiotech|Science Lite|TheBrightsRussia|Future Foundation|LessWrong|Physics-Uspekhi|Сообщество молодых учёных|Первая Научно-Популярная Библиотека|Elementy.ru|Лекс Кравецкий|Информационный центр атомной отрасли|42nStudio|Вячеслав Пуговкин|Алисолекторий|Eureka Project|Когнитека|Indicator_Ru|Андрей Коренко|КиберЛенинка|Moscow Science Slam|LAKHTA VIEW|Мослекторий|Турнир Трёх Наук|Pavel Borodin|Dissernet TV|Общество Скептиков|Центр Архэ|Курилка Гутенберга|15x4 Talks|НаукаPRO|Познаем Общаясь|Прогресс-Школа|Зануда|Лаборатория Научных Видео|АРХЭология знаний|КритМышь|Фонд Эволюция|Фонд Траектория|XX2 ВЕК|Лекториум|Set Up|ПостНаука|Alpha Centauri|Astro Channel|Открытый космос Зеленого кота|SpaceX по-русски|Космос Просто|Рафаэль и Ко.|Море ясности|Zемлякова|Улица Шкловского|Space Room|Sergei B. Popov|Телестудия Роскосмоса|Центр подготовки космонавтов имени Ю.А.Гагарина|ROSCOSMOS Media Store|Cosmos+ Астрономия для начинающих|Дмитрий Селезнев DS Astro Photography|Astromagazin|AstroScope|Mad Astronomer|Blackfieldspace|РКЦ - Прогресс|astro-alert|Живая Вселенная|Ввысь, в космос!|Mocковский Астрономический Клуб|Вселенная Пространство Время|Небо - не предел|Александр Мисуркин|Den Efremov|Filipp Terekhov|Астрономия в Серпухове|Макар Светлый|Физика от Побединского|ensemb|Физтех.Science|ПРОСТАЯ НАУКА|Игорь Белецкий|Проект о Радиации - Радиоактивные материалы|Занимательная физика в Городненской ОШ|Огненное ТВ|Минута Физики|HI-TESTING|EmpiricSchool|Закройте Дует|Halyk Smart|eNano|Alexander Olikevich|Химия – Просто|Thoisoi|ChemistryToday|ДУСТХИМХАБРПРОМ опыты по химии|RUSCHIMIK Канал о химии|Skeptik|Научное шоу профессора Николя|LiFeUP 331|Oleg Grigoryev|Наука детям|Александр Панчин|Azureim|Всё как у зверей|Живая Планета|КАЖДОЙ ТВАРИ ПО ПАРЕ|Bioreactor|FutureCollector|Aleksandra LearnBiology|Борис Цацулин|ПО ФАКТАМ|Тайны ДНК|Topanatomy|Microbia|AgroPrognoz|Denis A.Timofeev|Научная Среда|Частный экзотариум|АНТРОПОГЕНЕЗ РУ|Redroom Studio|Упоротый Палеонтолог|IndiansWorld.org|uchilka_istorii|Arzamas|elpositive|Политехнический музей|Музей Экспериментаниум|Государственный Дарвиновский музей|Московский Планетарий|Большой новосибирский планетарий|Ижевский планетарий|Палеонтологический музей имени Ю.А. Орлова|Музей Космонавтики"""
    s = s.lower()
    s = re.compile(s)
    acc = []
    for field in users:
        acc.append(bool(s.match(str(user[field]).lower())))
    for _, subscribtion in user_sub.iterrows():
        acc.append(bool(s.match(str(subscribtion['full_name']).lower())))
        acc.append(bool(s.match(str(subscribtion['shrot_name']).lower())))
    acc = sum(map(lambda x: int(x), acc))
    return acc > threshold

In [5]:
def sport_filter(user, user_sub, threshold=6):
    s = """.*\sматч\s.*|.*\ssport\s.*|.*\sспорт.*|.*здоровое питание.*|.*\sпитание.*|.*\sрацион\s.*|.*прием пищи.*|.*\sсушка\s.*|.*\sтренировк.*|.*\sкачалка.*|.*спортзал.*|.*фитнес.*|.*\sgym\s.*|.*\sсальто\s.*|.*баскетбол.*|.*\sбокс\s.*|.*\sборьба\s.*|.*\sсамбо\s.*|.*\sвольная\s.*|.*\sгреко-римская\s.*|.*\sвелоспорт\s.*|.*\sтрековый\s.*|.*\sмаунтинбайк\s.*|.*\sшоссейный\s.*|.*\sводные виды спорта\s.*|.*\sводное\s.*|.*\sпрыжки в\s.*|.*\sсинхронное\s.*|.*\sплавание\s.*|.*\sволейбол\s.*|.*\sпляжный\s.*|.*\sгандбол\s.*|.*\sгимнастика.*|.*прыжки на.*|.*спортивная.*|.*художественная.*|.*гольф.*|.*гребля на байдарках и каноэ.*|.*гребля.*|.*дзюдо.*|.*конный спорт.*|.*лёгкая атлетика.*|.*настольный теннис.*|.*парусный спорт.*|.*регби.*|.*современное пятиборье.*|.*стрельба.*|.*стрельба из лука.*|.*теннис.*|.*триатлон.*|.*тхэквондо.*|.*тяжёлая атлетика.*|.*фехтование.*|.*\sфутбол\s.*|.*хоккей на траве.*|.*баскетбол.*|.*баскетбол.*|.*велоспорт.*|.*бейсбол.*|.*спортивное скалолазание.*|.*\sкарате\s.*|.*скейтбординг.*|.*сёрфинг.*|.*/sufc/s.*"""
    s = s.lower()
    s = re.compile(s)
    acc = []
    for field in users:
        match = s.match(str(user[field]).lower())
        if match:
            acc.append(match)
    for _, subscribtion in user_sub.iterrows():
        full = s.match(str(subscribtion['full_name']).lower())
        short = s.match(str(subscribtion['shrot_name']).lower())
        if full:
            acc.append(full)
        if short:
            acc.append(short)
    acc = sum(map(lambda x: int(bool(x)), acc))
    return acc > threshold

In [6]:
def obzora_filter(user, user_sub, threshold=5):
    s = """.*\sеда.*|.*\sкулинар.*|.*\sготовка.*|.*\sприготов.*|.*\sеду.*|.*\sаппетит.*|.*\sсыт.*|.*\sпища.*|.*\sужин.*|.*\sобед.*|.*\sзавтрак.*|.*\sвкус.*|.*\sресторан.*|.*\sзаказ еды.*|.*доставка еды.*|.*\sрецепты.*|.*\sсъесть.*|.*\sтрапеза.*|.*\sяндекс.еда.*|.*\sdelivery club.*|.*\sпицца\s.*|.*\sсуши\s.*|.*\sвок\s.*|.*\swok\s.*|.*\sобжор.*|.*\sпохуде.*|.*\sвес\s.*"""
    s = s.lower()
    s = re.compile(s)
    acc = []
    for field in users:
        match = s.match(str(user[field]).lower())
        if match:
            acc.append(match)
    for _, subscribtion in user_sub.iterrows():
        full = s.match(str(subscribtion['full_name']).lower())
        short = s.match(str(subscribtion['shrot_name']).lower())
        if full:
            acc.append(full)
        if short:
            acc.append(short)
    print(user['user_id'])
    print(acc)
    acc = sum(map(lambda x: int(bool(x)), acc))
    return acc > threshold

In [7]:
def travel_filter(user, user_sub, threshold=5):
    s = '''.*\sпутешеств.*|.*\sпутевки.*|.*\svandrouki.*|.*\sржд\s.*|.*\sавия.*|.*\sтеплоход.*|.*\sавтостоп.*|.*\strip\s.*|.*\sотпуск.*|.*\sулетел.*|.*\sслетал.*|.*билеты.*|.*\sперелеты.*|.*\sпереезды.*|.*booking.*|.*\sотель\s.*|.*couchsurfing.*|.*\sтур\s.*|.*\sзаграни.*|.*\sвиза\s.*|.*\sпосольство.*|.*\sязык\s.*|.*\sультура.*|.*\sсвет\s.*|.*\sземной шар.*|.*\sбортпроводник.*|iknow.travel|Страна.ру|Моя Планета|Аirpano|TripAdvisor|Redigo|Travel.ru|Тонкости туризма|Турбина.ру|Туристер|МАРШРУТЫ|РБК.Travel|Википутешествие|Автопутешествия по России|TURIZM.RU|Localway|РЖД|Tutu.ru|Hotellook|Booking.com|Expedia|TopHotels|KAYAK|AirBnb|Сouchsurfing|Hostelworld|Клуб путешествий Special|Клуб путешествий Михаила Кожухова|Poseidon Expeditions|Ультра|50 авторских туров|Gfhome|Экстремальный портал|Lovozero|Adrenalinetour|Страны безвизового въезда|Lonely Planet|Вокруг Света|Полиглот|Дорлинг Киндерсли|Афиша|CRUISESTRAVEL|CruClub|Круизный Дом «МК»|PAC GROUP|Инфлот|Rentalcars|Economy Car Rentals|Hertz|Europcar Russia|Aviasales|поисковиков.|Skyscanner|Anywayanyday|Momondo|OZON.travel|Форум Винского|Русский Backpacker|Skitalets|Турбина|TripAdvisor|Вокруг света|GEO|National Geоgraphic|Афиша|За рубежом|Discovery|Conde Nast Traveller|TravelJay|TOTAL ESCAPE|Турист'''
    s = s.lower()
    s = re.compile(s)
    acc = []
    for field in users:
        match = s.match(str(user[field]).lower())
        if match:
            acc.append(match)
    for _, subscribtion in user_sub.iterrows():
        full = s.match(str(subscribtion['full_name']).lower())
        short = s.match(str(subscribtion['shrot_name']).lower())
        if full:
            acc.append(full)
        if short:
            acc.append(short)
    acc = sum(map(lambda x: int(bool(x)), acc))
    return acc > threshold

In [8]:
def bank_filter(user, user_sub, threshold=0):
    s = """.*\sбанк\s.*|.*\sАбсолют\s.*|.*\sАзиатско-тихоокеанский банк.*|.*\sАк барс\s.*|.*\sАлмазэргиэнбанк\s.*|.*\sАльфа-банк\s.*|.*\sБалтийский банк\s.*|.*\sБалтинвестбанк\s.*|.*\sОткрытие\s.*|.*\sВТБ.*|.*\sкредит свисс.*|.*\sБанк Москвы.*|.*\sВозрождение банк\s.*|.*\sВосточный банк\s.*|.*\sВсероссийский банк развития регионов.*|.*\sГазпромбанк.*|.*\sГута-банк.*|.*\sДальневосточный банк.*|.*\sДельтаКредит.*|.*\sДжии мани банк.*|.*\sЕврофинанс Моснарбанк.*|.*\sЗапсибкомбанк.*|.*\sЗенит банк\s.*|.*\sИвсельбанк.*|.*\sИнвестторгбанк.*|.*\sИнтеза\s.*|.*\sКольцо Урала\s.*|.*\sКредит Европа банк.*|.*\sКубань Кредит.*|.*\sКузнецкбизнесбанк.*|.*\sЛоко-банк.*|.*\sМеталлинвестбанк.*|.*\sМеткомбанк.*|.*\sМир бизнес банк.*|.*\sМосковский индустриальный банк.*|.*\sМосковский кредитный банк.*|.*\sМоскоммерцбанк.*|.*\sМособлбанк.*|.*\sМТС Банк.*|.*\sНациональный резервный банк.*|.*\sНовикомбанк.*|.*\sНордеа банк.*|.*\sПойдём!.*|.*\sПочта Банк.*|.*\sПрио-Внешторгбанк.*|.*\sПромсвязьбанк.*|.*\sПромТрансБанк.*|.*\sПромышленно-строительный банк.*|.*\sРайффайзенбанк.*|.*\sРенессанс Капитал.*|.*\sРенессанс Кредит.*|.*\sРосбанк.*|.*\sРосгосстрах банк.*|.*\sРоссия банк.*|.*\sРусфинанс банк.*|.*\sСбербанк.*|.*\sСвязь-банк.*|.*\sСибнефтебанк.*|.*\sСитибанк.*|.*\sСКБ\s.*|.*\sСМП\s.*|.*\sСобинбанк.*|.*\sСовкомбанк.*|.*\sСолидарность.*|.*\sСоюз банк\s.*|.*\sТинькофф.*|.*\sТранскапиталбанк.*|.*\sУглеметбанк.*|.*\sУралсиб банк.*|.*\sУральский банк реконструкции и развития.*|.*\sФИНАМ\s.*|.*\sФинсервис.*|.*\sХКФ банк.*|.*\sХлынов банк.*|.*\sЦентр-инвест.*|.*\sЧелиндбанк.*|.*\sЭнергомашбанк.*|.*\sЮникредит банк.*|.*\sЮнистрим.*"""
    s = s.lower()
    s = re.compile(s)
    acc = []
    for field in users:
        match = s.match(str(user[field]).lower())
        if match:
            acc.append(match)
    if acc:
        print(acc[0].group(0))
    acc = sum(map(lambda x: int(bool(x)), acc))
    return acc > threshold

In [9]:
def top_univer_filter(user, user_sub, threshold=0):
    s = """МГУ|МФТИ|НИЯУ МИФИ|СПбГУ|НИУ ВШЭ (ГУ-ВШЭ)|МГИМО МИД России|ТПУ|НГУ|МГТУ им. Н. Э. Баумана|СПбПУ Петра Великого (Политех)|РАНХиГС при Президенте РФ (АНХ при Правительстве РФ, РАГС при Президенте РФ)|УрФУ им. первого Президента России Б. Н. Ельцина|ТГУ|Финансовый университет при Правительстве РФ (бывш. ФА при Правительстве РФ)|Университет ИТМО|КФУ (бывш. КГУ им. Ульянова-Ленина)|ИЕиГН СФУ (бывш. КрасГУ)|НИТУ МИСиС|РГУ нефти и газа (НИУ) им. И. М. Губкина|РУДН|РЭУ им. Г. В. Плеханова|Первый МГМУ им. И. М. Сеченова (Сеченовский Университет) (бывш. ММА им. Сеченова)|РНИМУ (бывш. РГМУ) им. Н. И. Пирогова|ПСПбГМУ им. акад. Павлова|НИУ МЭИ|ННГУ им. Н. И. Лобачевского|МАИ|ДВФУ|ВАВТ|ЮФУ (бывш. РГУ)|НГТУ (НЭТИ)|МГЛУ (бывш. МГПИИЯ им. Мориса Тореза)|СПбГЭУ|СВФУ им. М. К. Аммосова (бывш. ЯГУ)|СГАУ им. Королева|МГЮА (Университет им. О. Е. Кутафина)|СибГМУ (ТМИ)|КГМУ|СПбГЭТУ (ЛЭТИ)|МГСУ НИУ (МГСУ-МИСИ)|КНИТУ-КАИ им. А. Н. Туполева|МНИОИ им. П. А. Герцена |ВГУ|Горный университет (бывш. СПГГУ)|ТУСУР (бывш. ТАСУР, ТИАСУР)|СамГМУ|СЗГМУ им. Мечникова (бывш. СПбГМА, СПбМАПО, ЛСГМИ) |РГГУ|МПГУ (бывш. МГПИ им. Ленина)|АлтГУ|НИУ БелГУ|СамГТУ|Университет «Дубна»"""
    s = s.lower()
    s = re.compile(s)
    acc = []
    match = s.match(str(user['university_name']).lower())
    if match:
        acc.append(match)
    acc = sum(map(lambda x: int(bool(x)), acc))
    return acc > threshold

In [10]:
def in_rus_city(user, user_sub, threshold=0):
    s = """Абаза|Абакан|Абдулино|Абинск|Агидель|Агрыз|Адыгейск|Азнакаево|Азов|Ак-Довурак|Аксай|Алагир|Алапаевск|Алатырь|Алдан|Алейск|Александров|Александровск|Александровск-Сахалинский|Алексеевка|Алексин|Алзамай|Алупка|Алушта|Альметьевск|Амурск|Анадырь|Анапа|Ангарск|Андреаполь|Анжеро-Судженск|Анива|Апатиты|Апрелевка|Апшеронск|Арамиль|Аргун|Ардатов|Ардон|Арзамас|Аркадак|Армавир|Армянск|Армянськ|Арсеньев|Арск|Артем|Артемовск|Артемовский|Архангельск|Асбест|Асино|Астрахань|Аткарск|Ахтубинск|Ахтубинск-7|Ачинск|Аша|Бабаево|Бабушкин|Бавлы|Багратионовск|Байкальск|Баймак|Бакал|Баксан|Балабаново|Балаково|Балахна|Балашиха|Балашов|Балей|Балтийск|Барабинск|Барнаул|Барыш|Батайск|Бахчисарай|Бежецк|Белгород|Белебей|Белев|Белинский|Белово|Белогорск|Белогорск|Белозерск|Белокуриха|Беломорск|Белорецк|Белореченск|Белоусово|Белоярский|Белый|Бердск|Березники|Березовский|Березовский|Беслан|Бийск|Бикин|Билибино|Биробиджан|Бирск|Бирюсинск|Бирюч|Благовещенск|Благовещенск|Благодарный|Бобров|Богданович|Богородицк|Богородск|Боготол|Богучар|Бодайбо|Бокситогорск|Болгар|Бологое|Болотное|Болохово|Болхов|Большой Камень|Бор|Борзя|Борисоглебск|Боровичи|Боровск|Боровск-1|Бородино|Братск|Бронницы|Брянск|Бугульма|Бугуруслан|Буденновск|Бузулук|Буинск|Буй|Буйнакск|Бутурлиновка|Валдай|Валуйки|Велиж|Великие Луки|Великие Луки-1|Великий Новгород|Великий Устюг|Вельск|Венев|Верещагино|Верея|Верхнеуральск|Верхний Тагил|Верхний Уфалей|Верхняя Пышма|Верхняя Салда|Верхняя Тура|Верхотурье|Верхоянск|Весьегонск|Ветлуга|Видное|Вилюйск|Вилючинск|Вихоревка|Вичуга|Владивосток|Владикавказ|Владимир|Волгоград|Волгодонск|Волгореченск|Волжск|Волжский|Вологда|Володарск|Волоколамск|Волосово|Волхов|Волчанск|Вольск|Вольск-18|Воркута|Воронеж|Воронеж-45|Ворсма|Воскресенск|Воткинск|Всеволожск|Вуктыл|Выборг|Выкса|Высоковск|Высоцк|Вытегра|Вышний Волочек|Вяземский|Вязники|Вязьма|Вятские Поляны|Гаврилов Посад|Гаврилов-Ям|Гагарин|Гаджиево|Гай|Галич|Гатчина|Гвардейск|Гдов|Геленджик|Георгиевск|Глазов|Голицыно|Горбатов|Горно-Алтайск|Горнозаводск|Горняк|Городец|Городище|Городовиковск|Городской округ Черноголовка|Гороховец|Горячий Ключ|Грайворон|Гремячинск|Грозный|Грязи|Грязовец|Губаха|Губкин|Губкинский|Гудермес|Гуково|Гулькевичи|Гурьевск|Гурьевск|Гусев|Гусиноозерск|Гусь-Хрустальный|Давлеканово|Дагестанские Огни|Далматово|Дальнегорск|Дальнереченск|Данилов|Данков|Дегтярск|Дедовск|Демидов|Дербент|Десногорск|Джанкой|Джанкой|Дзержинск|Дзержинский|Дивногорск|Дигора|Димитровград|Дмитриев|Дмитров|Дмитровск|Дно|Добрянка|Долгопрудный|Долинск|Домодедово|Донецк|Донской|Дорогобуж|Дрезна|Дубна|Дубовка|Дудинка|Духовщина|Дюртюли|Дятьково|Евпатория|Егорьевск|Ейск|Екатеринбург|Елабуга|Елец|Елизово|Ельня|Еманжелинск|Емва|Енисейск|Ермолино|Ершов|Ессентуки|Ефремов|Железноводск|Железногорск|Железногорск|Железногорск-Илимский|Железнодорожный|Жердевка|Жигулевск|Жиздра|Жирновск|Жуков|Жуковка|Жуковский|Завитинск|Заводоуковск|Заволжск|Заволжье|Задонск|Заинск|Закаменск|Заозерный|Заозерск|Западная Двина|Заполярный|Зарайск|Заречный|Заречный|Заринск|Звенигово|Звенигород|Зверево|Зеленогорск|Зеленогорск|Зеленоград|Зеленоградск|Зеленодольск|Зеленокумск|Зерноград|Зея|Зима|Златоуст|Злынка|Змеиногорск|Знаменск|Зубцов|Зуевка|Ивангород|Иваново|Ивантеевка|Ивдель|Игарка|Ижевск|Избербаш|Изобильный|Иланский|Инза|Инкерман|Инсар|Инта|Ипатово|Ирбит|Иркутск|Иркутск-45|Исилькуль|Искитим|Истра|Истра-1|Ишим|Ишимбай|Йошкар-Ола|Кадников|Казань|Калач|Калачинск|Калач-на-Дону|Калининград|Калининск|Калтан|Калуга|Калязин|Камбарка|Каменка|Каменногорск|Каменск-Уральский|Каменск-Шахтинский|Камень-на-Оби|Камешково|Камызяк|Камышин|Камышлов|Канаш|Кандалакша|Канск|Карабаново|Карабаш|Карабулак|Карасук|Карачаевск|Карачев|Каргат|Каргополь|Карпинск|Карталы|Касимов|Касли|Каспийск|Катав-Ивановск|Катайск|Качканар|Кашин|Кашира|Кашира-8|Кедровый|Кемерово|Кемь|Керчь|Кизел|Кизилюрт|Кизляр|Кимовск|Кимры|Кингисепп|Кинель|Кинешма|Киреевск|Киренск|Киржач|Кириллов|Кириши|Киров|Киров|Кировград|Кирово-Чепецк|Кировск|Кировск|Кирс|Кирсанов|Киселевск|Кисловодск|Климовск|Клин|Клинцы|Княгинино|Ковдор|Ковров|Ковылкино|Когалым|Кодинск|Козельск|Козловка|Козьмодемьянск|Кола|Кологрив|Коломна|Колпашево|Колпино|Кольчугино|Коммунар|Комсомольск|Комсомольск-на-Амуре|Конаково|Кондопога|Кондрово|Константиновск|Копейск|Кораблино|Кореновск|Коркино|Королев|Короча|Корсаков|Коряжма|Костерево|Костомукша|Кострома|Котельники|Котельниково|Котельнич|Котлас|Котово|Котовск|Кохма|Красавино|Красноармейск|Красноармейск|Красновишерск|Красногорск|Краснодар|Красное Село|Краснозаводск|Краснознаменск|Краснознаменск|Краснокаменск|Краснокамск|Красноперекопск|Красноперекопск|Краснослободск|Краснослободск|Краснотурьинск|Красноуральск|Красноуфимск|Красноярск|Красный Кут|Красный Сулин|Красный Холм|Кременки|Кронштадт|Кропоткин|Крымск|Кстово|Кубинка|Кувандык|Кувшиново|Кудымкар|Кузнецк|Кузнецк-12|Кузнецк-8|Куйбышев|Кулебаки|Кумертау|Кунгур|Купино|Курган|Курганинск|Курильск|Курлово|Куровское|Курск|Куртамыш|Курчатов|Куса|Кушва|Кызыл|Кыштым|Кяхта|Лабинск|Лабытнанги|Лагань|Ладушкин|Лаишево|Лакинск|Лангепас|Лахденпохья|Лебедянь|Лениногорск|Ленинск|Ленинск-Кузнецкий|Ленск|Лермонтов|Лесной|Лесозаводск|Лесосибирск|Ливны|Ликино-Дулево|Липецк|Липки|Лиски|Лихославль|Лобня|Лодейное Поле|Ломоносов|Лосино-Петровский|Луга|Луза|Лукоянов|Луховицы|Лысково|Лысьва|Лыткарино|Льгов|Любань|Люберцы|Любим|Людиново|Лянтор|Магадан|Магас|Магнитогорск|Майкоп|Майский|Макаров|Макарьев|Макушино|Малая Вишера|Малгобек|Малмыж|Малоархангельск|Малоярославец|Мамадыш|Мамоново|Мантурово|Мариинск|Мариинский Посад|Маркс|Махачкала|Мглин|Мегион|Медвежьегорск|Медногорск|Медынь|Межгорье|Междуреченск|Мезень|Меленки|Мелеуз|Менделеевск|Мензелинск|Мещовск|Миасс|Микунь|Миллерово|Минеральные Воды|Минусинск|Миньяр|Мирный|Мирный|Михайлов|Михайловка|Михайловск|Михайловск|Мичуринск|Могоча|Можайск|Можга|Моздок|Мончегорск|Морозовск|Моршанск|Мосальск|Москва|Московский|Московский|Муравленко|Мураши|Мурманск|Муром|Мценск|Мыски|Мытищи|Мышкин|Набережные Челны|Навашино|Наволоки|Надым|Назарово|Назрань|Называевск|Нальчик|Нариманов|Наро-Фоминск|Нарткала|Нарьян-Мар|Находка|Невель|Невельск|Невинномысск|Невьянск|Нелидово|Неман|Нерехта|Нерчинск|Нерюнгри|Нестеров|Нефтегорск|Нефтекамск|Нефтекумск|Нефтеюганск|Нея|Нижневартовск|Нижнекамск|Нижнеудинск|Нижние Серги|Нижние Серги-3|Нижний Ломов|Нижний Новгород|Нижний Тагил|Нижняя Салда|Нижняя Тура|Николаевск|Николаевск-на-Амуре|Никольск|Никольск|Никольское|Новая Ладога|Новая Ляля|Новоалександровск|Новоалтайск|Новоаннинский|Нововоронеж|Новодвинск|Новозыбков|Новокубанск|Новокузнецк|Новокуйбышевск|Новомичуринск|Новомосковск|Новопавловск|Новоржев|Новороссийск|Новосибирск|Новосиль|Новосокольники|Новотроицк|Новоузенск|Новоульяновск|Новоуральск|Новохоперск|Новочебоксарск|Новочеркасск|Новошахтинск|Новый Оскол|Новый Уренгой|Ногинск|Нолинск|Норильск|Ноябрьск|Нурлат|Нытва|Нюрба|Нягань|Нязепетровск|Няндома|Облучье|Обнинск|Обоянь|Обь|Одинцово|Ожерелье|Озерск|Озерск|Озеры|Октябрьск|Октябрьский|Окуловка|Олекминск|Оленегорск|Оленегорск-1|Оленегорск-2|Оленегорск-4|Олонец|Омск|Омутнинск|Онега|Опочка|Орёл|Оренбург|Орехово-Зуево|Орлов|Орск|Оса|Осинники|Осташков|Остров|Островной|Острогожск|Отрадное|Отрадный|Оха|Оханск|Очер|Павлово|Павловск|Павловск|Павловский Посад|Палласовка|Партизанск|Певек|Пенза|Первомайск|Первоуральск|Перевоз|Пересвет|Переславль-Залесский|Пермь|Пестово|Петергоф|Петров Вал|Петровск|Петровск-Забайкальский|Петрозаводск|Петропавловск-Камчатский|Петухово|Петушки|Печора|Печоры|Пикалево|Пионерский|Питкяранта|Плавск|Пласт|Плес|Поворино|Подгорное|Подольск|Подпорожье|Покачи|Покров|Покровск|Полевской|Полесск|Полысаево|Полярные Зори|Полярный|Поронайск|Порхов|Похвистнево|Почеп|Починок|Пошехонье|Правдинск|Приволжск|Приморск|Приморск|Приморско-Ахтарск|Приозерск|Прокопьевск|Пролетарск|Протвино|Прохладный|Псков|Пугачев|Пудож|Пустошка|Пучеж|Пушкин|Пушкино|Пущино|Пыталово|Пыть-Ях|Пятигорск|Радужный|Радужный|Райчихинск|Раменское|Рассказово|Ревда|Реж|Реутов|Ржев|Родники|Рославль|Россошь|Ростов|Ростов-на-Дону|Рошаль|Ртищево|Рубцовск|Рудня|Руза|Рузаевка|Рыбинск|Рыбное|Рыльск|Ряжск|Рязань|Саки|Саки|Салават|Салаир|Салехард|Сальск|Самара|Санкт-Петербург|Саранск|Сарапул|Саратов|Саров|Сасово|Сатка|Сафоново|Саяногорск|Саянск|Светлогорск|Светлоград|Светлый|Светогорск|Свирск|Свободный|Себеж|Севастополь|Северобайкальск|Северодвинск|Северо-Курильск|Североморск|Североуральск|Северск|Севск|Сегежа|Сельцо|Семенов|Семикаракорск|Семилуки|Сенгилей|Серафимович|Сергач|Сергиев Посад|Сергиев Посад-7|Сердобск|Серов|Серпухов|Сертолово|Сестрорецк|Сибай|Сим|Симферополь|Сковородино|Скопин|Славгород|Славск|Славянск-на-Кубани|Сланцы|Слободской|Слюдянка|Смоленск|Снегири|Снежинск|Снежногорск|Собинка|Советск|Советск|Советск|Советская Гавань|Советский|Сокол|Солигалич|Соликамск|Солнечногорск|Солнечногорск-2|Солнечногорск-25|Солнечногорск-30|Солнечногорск-7|Сольвычегодск|Соль-Илецк|Сольцы|Сольцы 2|Сорочинск|Сорск|Сортавала|Сосенский|Сосновка|Сосновоборск|Сосновый Бор|Сосногорск|Сочи|Спас-Деменск|Спас-Клепики|Спасск|Спасск-Дальний|Спасск-Рязанский|Среднеколымск|Среднеуральск|Сретенск|Ставрополь|Старая Купавна|Старая Русса|Старица|Стародуб|Старый крым|Старый Оскол|Стерлитамак|Стрежевой|Строитель|Струнино|Ступино|Суворов|Судак|Суджа|Судогда|Суздаль|Суоярви|Сураж|Сургут|Суровикино|Сурск|Сусуман|Сухиничи|Сухой Лог|Сызрань|Сыктывкар|Сысерть|Сычевка|Сясьстрой|Тавда|Таганрог|Тайга|Тайшет|Талдом|Талица|Тамбов|Тара|Тарко-Сале|Таруса|Татарск|Таштагол|Тверь|Теберда|Тейково|Темников|Темрюк|Терек|Тетюши|Тимашевск|Тихвин|Тихорецк|Тобольск|Тогучин|Тольятти|Томари|Томмот|Томск|Топки|Торжок|Торопец|Тосно|Тотьма|Трехгорный|Трехгорный-1|Троицк|Троицк|Трубчевск|Туапсе|Туймазы|Тула|Тулун|Туран|Туринск|Тутаев|Тында|Тырныауз|Тюкалинск|Тюмень|Уварово|Углегорск|Углич|Удачный|Удомля|Ужур|Узловая|Улан-Удэ|Ульяновск|Унеча|Урай|Урень|Уржум|Урус-Мартан|Урюпинск|Усинск|Усмань|Усолье|Усолье-Сибирское|Уссурийск|Усть-Джегута|Усть-Илимск|Усть-Катав|Усть-Кут|Усть-Лабинск|Устюжна|Уфа|Ухта|Учалы|Уяр|Фатеж|Феодосия|Фокино|Фокино|Фролово|Фрязино|Фурманов|Хабаровск|Хадыженск|Ханты-Мансийск|Харабали|Харовск|Хасавюрт|Хвалынск|Хилок|Химки|Холм|Холмск|Хотьково|Цивильск|Цимлянск|Чадан|Чайковский|Чапаевск|Чаплыгин|Чебаркуль|Чебоксары|Чегем|Чекалин|Челябинск|Чердынь|Черемхово|Черепаново|Череповец|Черкесск|Чермоз|Черноголовка|Черногорск|Чернушка|Черняховск|Чехов|Чехов-2|Чехов-3|Чехов-8|Чистополь|Чита|Чкаловск|Чудово|Чулым|Чулым-3|Чусовой|Чухлома|Шагонар|Шадринск|Шали|Шарыпово|Шарья|Шатура|Шахтерск|Шахты|Шахунья|Шацк|Шебекино|Шелехов|Шенкурск|Шилка|Шимановск|Шиханы|Шлиссельбург|Шумерля|Шумиха|Шуя|Щекино|Щелкино|Щелково|Щербинка|Щигры|Щучье|Электрогорск|Электросталь|Электроугли|Элиста|Энгельс|Энгельс-19|Энгельс-2|Эртиль|Юбилейный|Югорск|Южа|Южно-Сахалинск|Южно-Сухокумск|Южноуральск|Юрга|Юрьевец|Юрьев-Польский|Юрюзань|Юхнов|Юхнов-1|Юхнов-2|Ядрин|Якутск|Ялта|Ялуторовск|Янаул|Яранск|Яровое|Ярославль|Ярцево|Ясногорск|Ясный|Яхрома"""
    s = s.lower()
    s = re.compile(s)
    acc = []
    match = s.match(str(user['city']).lower())
    if match:
        acc.append(match)
    acc = sum(map(lambda x: int(bool(x)), acc))
    return acc > threshold

In [11]:
def live_in_rus(user, user_sub, threshold=0):
    return str(user['country']).lower() == 'россия'

In [12]:
def anime_filter(user, user_sub, threshold=0):
    s = """.*\sаниме\s.*|.*\sанимэ\s.*|.*\sanime\s.*|.*\sjapan multfilms\s.*|.*\sjapan.*\s|.*\sяпон.*\s|Наруто|Блич|Хвост феи|Учитель Мафиози Реборн|Темный дворецкий|Темней черного|Ангельские ритмы|Free|За гранью|Pokemon!|Ван Пис|Синий экзорцист|Тетрадь смерти|Врата Штайнера|Атака титанов|Дюрарара|Адская девочка|Иная|Эльфийская песнь|Поющий принц|Сердца пандоры|Президент студсовета горничная|Бездомный бог|S.A.O|Амнезия|Волчица и пряности|Баскетбол Куроко|Актеры города Мекаку|Ходячий замок|Унесенные призраками|Мой сосед Тоторо|Принцесса Мононоке|Kill a Kill|Поцелуй сестер|История подделок|Скрытое сокровище Нананы|Черная пуля|Забавы богов|И все таки мир прекрасен|Клинок и душа|Друзья на неделю|Непутевый ученик в школе магии|Беспокойные сердца|Нана|Проект Кей|Нет игры, нет жизни|У меня мало друзей|Аватар: Легенда об Аанге|Аватар:Легенда о Корре|Приключение старшеклассников|Золотой парень|Гинтама|Кровь - С|Демоны старшей школы|Школа мертвяков|Hyoka тебе не уйти!|Семь обличий Ямато Надэсико|Озорной поцелуй|Розарио + вампир|Сатана на подработке|Пожиратели душ|Школьные войны|Хеллсинг|Афросамурай|Шаман Кинг|Торадора|Христовый поход Хроно|K-on|Загадка истории дьявола|Притворная любовь|Евангелион|Маги: Лабиринт волшебства|Соник|Haikyuu!!! - Волейбол|Путь Асса|5 сантиметров в секунду|Девочка покорившая время|В лесу мерцания светлячков|Блокнот Бога|Мы ждем тебя летом|Мангака и все все все|Скажи я люблю тебя|Прикончи меня, детка|Отвязная троица|Цикл историй|Удар крови|Розовая пора моей школьной жизни сплошной обман|Буря Потерь|Инари, лисицы и волшебная любовь|Не моя вина, что я не популярна|No 6|Самурай, который пахнет подсолнухами|Очень приятно, Бог|Лаборатория любви|Безоблачное завтра|Дьявольские возлюбленные|Дружеский комплекс|Лучшая в мире первая любовь|Пингвиний барабан|Пчелиная почта|История монстров|Конфликт братьев|Рынок Тамако|Дети Индиго|Рандеву с жизнью|Брунгильда во тьме|Санка Рэя|Детективное агентво Хаматора|Апокалипсис|Бтуум|Карнавал|Мастер Муши|Вельзевул|Токийские вороны|Макен-ки|Сломанный меч|Логин - горизонт|Черная сталь|Мы из общаги "Кавай"|Война магов|Трусливый велосипедист|Зараженный селектор|Нам нет пути назад|Опровергающая пуля|Монстр за соседней партой|История семьи Аркана|Аполлон - дети на холме|Ведьма за работой|Стрелок с черной скалы|Город, в котором тв живешь|В лабиринте чужеземных перекрестков|Подвиг школьного совета|Унылый учитель|Рыцарь вампир|Трудности первой любви|Парадайз кисс|Моя малышка|Это были мы|Не сдавайся|Стальной алхимик|Самурай Х|Однажды в Токио|Моя богиня|Жаркое лето|Девушка и ее кот|Сакура война миров|Гравитация|Куклы паразиты|Цирк уродов|Её и его обстоятельства|Шепот сердца|Нищебог же|Мифический детектив Локи|Дневник будущего|Она подросток, но я ее люблю|Красный, белый черный, желтый|ШАФФЛ|Пес и маньячка|Поцелуй во тьме|Предательство знает мое имя|Девушка влюбилась в старшую сестру|Корона вины|Гуррен-Лаганн|Индекс волшебства|Оружейная школа|Исчезновение Харуки Судзумии|Глядя на полумесяц|Чемпион|Любовь, выборы и шоколад|Призрак в доспехах|Стимбой|Тяжелый металл|Гангрейв|Хроники крыльев|Код гиас|Бакуган|Черный кот|Владыка скрытого мира|Добро пожаловать в NHK!|Повелитель стихий|Судьба: ночь схватки|Планета Зэт|Хеталия из страны Оси|Робин охотница на ведьм|Детройт города металла|Леди против дворецких|Заморозка|Ахуитительный котенок|Мост в звездное небо|Меланхолия|Меланхолия Харуми Судзумии|Спец класс А|Василиск|Моя хранительница Химари|Князь тьмы с последней парты|Бакуман|Дотянуться до тебя|Инуяша|Шики|Волчий дождь|Тетрадь дружбы Нацумэ|Уж не зомби ли это|Демоны старшей школы|Дневник будущего|Дафна : тайна сияющих вод|12 королевств|Исток|Нанако|Стальная тревога|Devil my cry|Союз серокрылых|Код ангела|Самурай чампу|Волшебное лето|Эпоха смут|Охота на призраков|Собака и кошечка|Рыцарь Сидонии|Академия драгонаров|Заморозка (фризинг)|Вишневые шалости|Ваш заказ - это кролик?|Черный полюс Бронхильды|Члены школьного совета|Эстетика заблудшего героя|Академия оккультных наук|Фрактал|Штурмовые ведьмы|Моя сестренка в последнее время немного странная|Извращенка со вторым размером|Хочу быть сильнее всех|Клинок королевы : восстание|Буйство цветов:самурай невеста|Старшая школа: демоны против падших|Возьми меня!|Эй, люби меня всерьез|Ты-хозяин, я - слуга|Секрет дворецкого|Стигматы квайзеров|Девы гражданской войны: Розовый парадокс|Драконий кризис|Небо и земля: Глава о прошлом|Не приставай ко мне!|Махороматик: с возвращением!|Рыцари Гуларта: Пробудившееся темное начало|Он мой хозяин|Метка драконьего всадника|Инугами и Нэкояма|Капитан Планета|Академия драконьих наездников|Мастерская Эски и Локи"""
    s = s.lower()
    s = re.compile(s)
    acc = []
    for field in users:
        match = s.match(str(user[field]).lower())
        if match:
            acc.append(match)
    for _, subscribtion in user_sub.iterrows():
        full = s.match(str(subscribtion['full_name']).lower())
        short = s.match(str(subscribtion['shrot_name']).lower())
        if full:
            acc.append(full)
        if short:
            acc.append(short)
    acc = sum(map(lambda x: int(bool(x)), acc))
    return acc > threshold

In [13]:
def village_filter(user, user_sub, threshold=3):
    s = """.*\sдеревн.*|.*\sдеревенские\s.*"""
    s = s.lower()
    s = re.compile(s)
    acc = []
    for field in users:
        match = s.match(str(user[field]).lower())
        if match:
            acc.append(match)
    for _, subscribtion in user_sub.iterrows():
        full = s.match(str(subscribtion['full_name']).lower())
        short = s.match(str(subscribtion['shrot_name']).lower())
        if full:
            acc.append(full)
        if short:
            acc.append(short)
    acc = sum(map(lambda x: int(bool(x)), acc))
    return acc > threshold

In [15]:
#пробный запуск
for user in users.iterrows():
    user = user[1]
    user_sub = sub[sub['user_id'] == user['user_id']]
    if anime_filter(user, user_sub):
        print(f"vk.com/id{user['user_id']}")

vk.com/id84.0
vk.com/id166.0
vk.com/id169.0
vk.com/id260.0
vk.com/id323.0
vk.com/id398.0
vk.com/id473.0
vk.com/id530.0
vk.com/id536.0
vk.com/id657.0
vk.com/id721.0
vk.com/id879.0
vk.com/id1004.0
vk.com/id1010.0
vk.com/id1139.0
vk.com/id1219.0
vk.com/id1363.0
vk.com/id1439.0
vk.com/id1512.0
vk.com/id1564.0
vk.com/id1632.0
vk.com/id1635.0
vk.com/id1639.0
vk.com/id1700.0
vk.com/id1748.0
vk.com/id1749.0
vk.com/id1812.0
vk.com/id1817.0
vk.com/id1818.0
vk.com/id1883.0
vk.com/id1959.0
vk.com/id2087.0
vk.com/id2231.0
vk.com/id2235.0
vk.com/id2466.0
vk.com/id2626.0
vk.com/id2706.0
vk.com/id2847.0
vk.com/id2904.0
vk.com/id2908.0
vk.com/id2975.0
vk.com/id3006.0
vk.com/id3163.0
vk.com/id3166.0
vk.com/id3386.0
vk.com/id3646.0
vk.com/id3693.0
vk.com/id3696.0
vk.com/id3752.0
vk.com/id3757.0
vk.com/id3857.0
vk.com/id3858.0
vk.com/id3932.0
vk.com/id3994.0
vk.com/id4000.0
vk.com/id4049.0
vk.com/id4051.0
vk.com/id4208.0
vk.com/id4275.0
vk.com/id4423.0
vk.com/id4566.0
vk.com/id4604.0
vk.com/id4755.0
vk.co

KeyboardInterrupt: 